In [1]:
#This is the Notebook example for how to form files of Gauges and FGMAX for the GEOCLAW program
"""
Created By Cody Kesler and Hunter
Property of BYU Mathematics Dept.
"""

%matplotlib inline
%run gauge.py
import ShoreAngle
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
gauges = list()

In [2]:
#EXAMPLE GAUGE. COPY AND FILL IN FOR AS MANY GAUGES AS YOU HAVE:
city_name = 'Pulu Ai - Wichmann'
name = 10001 # Pulu Ai - Wichmann
longitude = -4.517863
latitude = 129.7745653
distance = 2.5 # in kilometers (max 5) //TODO Not sure what this is??
kind = ['skewnorm', 'norm', 'chi2']
#Depending on how kind is defined
# For kind = 'skewnorm'
arrival_skew_param = None
arrival_mean = None  # in minutes
arrival_std = None
arrival_params = [arrival_skew_param, arrival_mean, arrival_std]

# For kind = 'norm'
height_mean = 1. # in meters
height_std =  0.2
height_params = [height_mean, height_std]

# For kind = 'chi2'
inundation_k = None
inundation_lower_bound = None
inundation_params = [inundation_k, inundation_lower_bound]

# USE IF YOU HAVE SHORE PROFILES ANGLES IN AN EXCEL SHEET GENERATED FROM () :
#just the name of the excel file with no extention:
ShoreAngles = ShoreAngle.ShoreLineAngles('lonthor_profiles') 
profiles_to_average = [10,11,12,13]
LONTHOR_average_angle = ShoreAngles.getAveragesSlopeAngles(profiles_to_average)
beta = LONTHOR_average_angle

n = .03

g = Gauge(name, longitude, latitude, distance,
                kind, arrival_params, height_params, inundation_params, beta, n, city_name)
gauges.append(g.to_json())


#THIS IS FOR DISPLAYING THE DISTRIBUTIONS IF NEEDED:
#Skewnorm
domain = np.linspace(0,600,1000)
distrib = stats.skewnorm.pdf(domain, _skew_param, _mean,_std)
plt.plot(domain,distrib)
plt.xlabel("Appropriate Label")
plt.show()

# Normal Distribution
dist = stats.norm(_mean, _std)
domain = np.linspace(0,8,1000)
plt.plot(domain, dist.pdf(domain))
plt.xlabel("Appropriate Label")
plt.plot()
plt.show()

## Chi2
domain = np.linspace(_lower_bound, _upper_bound,100)
plt.plot(domain, stats.chi(height_k).pdf(domain - .5))
plt.xlabel("Appropriate Label")
plt.show()


Arrival/Height/Inundation Distribution Justification (FOR ABOVE): Explain why you chose the numbers you did

In [ ]:
#This is to save files to be used to input into GEOCLAW

#Puts the gauges into a numpy array file 
np.save('gauges', gauges)

#Formates the FG Max data needed for the GEOCLAW run 
#Adjust the numbers as needed
f = open('fgmax_grid.txt','w')
f.write('1.0000000000e+01            # tstart_max\n')
f.write('1.0000000000e+10            # tstart_max\n')
f.write('0.0000000000e+00            # dt_check\n')
f.write('3                           # min_level_check\n')
f.write('1.0000000000e-01            # arrival_tol\n')
f.write('0                           # point_style\n')
f.write(str(len(gauges)) + '                           # npts\n')


for gauge in gauges:
    f.write(str(gauge['latitude']) + '       ' + str(gauge['longitude']) + '\n')

f.close()